# Load Data

In [2]:
import os
import pandas as pd 
import numpy as np
import logging
import csv
import re
import logging
import optparse
import dedupe
from unidecode import unidecode
from itertools import product

## Setup

In [18]:
input_file = 'fodors_zagats_pypostal.csv'
output_file = 'fodors_zagats_pypostal_output.csv'
settings_file = 'fodors_zagats_learned_settings'
training_file = 'fodors_zagats_training.json'

In [19]:
fp = os.path.join(input_file)

## Dataframe view

In [21]:
input_df = pd.read_csv(fp, sep=',', quotechar='"')

In [8]:
categories = list(input_df['category'].unique())
categories = [x for x in categories if str(x) != 'nan']

In [9]:
#category_corpus = input_df[['name', 'category']].drop_duplicates().to_dict(orient='records')
category_corpus = input_df.drop_duplicates().to_dict(orient='records')

# Dedupe


## Import modules

In [12]:
def pre_process(val):
    """
    Do a little bit of data cleaning with the help of Unidecode and Regex.
    Things like casing, extra spaces, quotes and new lines can be ignored.
    """
    try:
        val = re.sub('  +', ' ', val)
        val = re.sub('\n', ' ', val)
        val = val.strip().strip('"').strip("'").lower().strip()
        # If data is missing, indicate that by setting the value to `None`
        if not val:
            val = None
    except Exception as e:
        print(e)
    return val

In [13]:
def get_clean_data_dict(file_path):
    data_d = {}
    with open(fp) as f:
        reader = csv.DictReader(f)
        for row in reader:
            clean_row = [(k, pre_process(v)) for (k, v) in row.items()]
            row_id = int(row['Id'])
            data_d[row_id] = dict(clean_row)

    return data_d

### Get Data in needed format

In [14]:
data_dict = get_clean_data_dict(fp)

### Define the Fields for dedupe


In [16]:
fields = [
    {'field' : 'name', 'type': 'Name'},
    {'field' : 'category', 
     'type': 'FuzzyCategorical',
     'categories': categories,
     'corpus': category_corpus,
     'has missing' : True},
    {'field' : 'name', 'type': 'String'},
    {'field' : 'address', 'type': 'Address'},
    {'field' : 'city', 'type': 'ShortString'},
    {'field' : 'phone', 'type': 'String'},
    {'field' : 'street', 'type': 'String', 'has missing' : True},
    {'field' : 'house_number', 'type': 'Exists', 'has missing' : True},
    {'field' : 'house', 'type': 'String', 'has missing' : True},   
]

### Instantiate Dedupe

In [17]:
deduper = dedupe.Dedupe(fields)

In [18]:
deduper.prepare_training(data_dict)

INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(LevenshteinCanopyPredicate: (2, house), SimplePredicate: (sameSevenCharStartPredicate, street))


In [19]:
dedupe.consoleLabel(deduper)

name : rainbow room
category : or 212/632-5100 american
address : 30 rockefeller plaza
city : new york
phone : 212/632-5000
street : None
house_number : None
house : 30 rockefeller plaza

name : rainbow room
category : american (new)
address : 30 rockefeller plaza
city : new york city
phone : 212-632-5000
street : None
house_number : None
house : 30 rockefeller plaza

0/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished


y


name : rose pistola
category : italian
address : 532 columbus avenue
city : san francisco
phone : 415/399-0499
street : columbus avenue
house_number : 532
house : None

name : rose pistola
category : italian
address : 532 columbus avenue
city : san francisco
phone : 415-399-0499
street : columbus avenue
house_number : 532
house : None

1/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


y


INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(PartialPredicate: (sameFiveCharStartPredicate, name, Surname), SimplePredicate: (sameFiveCharStartPredicate, house))
name : georgia grille
category : american
address : 2290 peachtree road peachtree square shopping center
city : atlanta
phone : 404/352-3517
street : peachtree road
house_number : 2290
house : square shopping center

name : georgia grille
category : southwestern
address : 2290 peachtree road
city : atlanta
phone : 404-352-3517
street : peachtree road
house_number : 2290
house : None

2/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


...

name : palace court
category : continental
address : 3570 las vegas blvd. s
city : las vegas
phone : 702/731-7547
street : las vegas blvd. s
house_number : 3570
house : None

name : empress court
category : asian
address : 3570 las vegas blvd. s
city : las vegas
phone : 702/731-7888
street : las vegas blvd. s
house_number : 3570
house : None

41/10 positive, 9/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


n


name : empress court
category : asian
address : 3570 las vegas blvd. s
city : las vegas
phone : 702/731-7888
street : las vegas blvd. s
house_number : 3570
house : None

name : palace court
category : french (new)
address : 3570 las vegas blvd. s.
city : las vegas
phone : 702-731-7110
street : las vegas blvd. s.
house_number : 3570
house : None

41/10 positive, 10/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


f


Finished labeling


In [20]:
deduper.train()

INFO:rlr.crossvalidation:using cross validation to find optimum alpha...
INFO:rlr.crossvalidation:optimum alpha: 0.100000, score 0.5545027521799167
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(TfidfTextCanopyPredicate: (0.4, name), TfidfTextCanopyPredicate: (0.4, phone))


In [33]:
# use a saved training file
# #with open(training_file, 'w') as tf:
#    deduper.writeTraining(tf)

In [34]:
# use saved settings file for duplication
#with open(settings_file, 'wb') as sf:
#    deduper.writeSettings(sf)

## run dedupe based on prior settings file

In [35]:
deduper = None
with open(settings_file, 'rb') as f:
    deduper = dedupe.StaticDedupe(f)

INFO:dedupe.api:((TfidfTextCanopyPredicate: (0.4, name), TfidfTextCanopyPredicate: (0.4, phone)),)


#### ------ 

In [36]:
threshold = deduper.threshold(data_dict, recall_weight=1)


INFO:dedupe.api:Maximum expected recall and precision
INFO:dedupe.api:recall: 0.986
INFO:dedupe.api:precision: 0.967
INFO:dedupe.api:With threshold: 0.544


In [37]:
clustered_dupes = deduper.match(data_dict, 0)

In [38]:
print('# duplicate sets', len(clustered_dupes))

# duplicate sets 112


In [39]:
cluster_membership = {}
cluster_id = 0
for (cluster_id, cluster) in enumerate(clustered_dupes):
    id_set, scores = cluster
    cluster_d = [data_dict[c] for c in id_set]
    canonical_rep = dedupe.canonicalize(cluster_d)
    for record_id, score in zip(id_set, scores):
        cluster_membership[record_id] = {
            "cluster id" : cluster_id,
            "canonical representation" : canonical_rep,
            "confidence": score
        }

In [52]:
singleton_id = cluster_id + 1
with open(output_file, 'w') as f_output, open(fp) as f_input:
    writer = csv.writer(f_output)
    reader = csv.reader(f_input)

    heading_row = next(reader)
    heading_row.insert(0, 'confidence_score')
    heading_row.insert(0, 'Cluster ID')
    canonical_keys = canonical_rep.keys()
    for key in canonical_keys:
        heading_row.append('canonical_' + key)

    writer.writerow(heading_row)

    for row in reader:
        row_id = int(row[0])
        if row_id in cluster_membership:
            cluster_id = cluster_membership[row_id]["cluster id"]
            canonical_rep = cluster_membership[row_id]["canonical representation"]
            row.insert(0, cluster_membership[row_id]['confidence'])
            row.insert(0, cluster_id)
            for key in canonical_keys:
                row.append(canonical_rep[key].encode('utf8'))
        else:
            row.insert(0, None)
            row.insert(0, singleton_id)
            singleton_id += 1
            for key in canonical_keys:
                row.append(None)
        writer.writerow(row)

# Predictions

In [54]:
df = pd.read_csv(output_file)

In [55]:
df.columns

Index(['Cluster ID', 'confidence_score', 'Id', 'source', 'name', 'category',
       'phone', 'city', 'address', 'street', 'house_number', 'house',
       'canonical_Id', 'canonical_source', 'canonical_name',
       'canonical_category', 'canonical_phone', 'canonical_city',
       'canonical_address', 'canonical_street', 'canonical_house_number',
       'canonical_house'],
      dtype='object')

In [3]:
import pandas as pd

In [ ]:
df = pd.read_csv(output_file)
df.sort_values(['Cluster ID'], inplace=True)
relevant_data = df[['Cluster ID', 'confidence_score', 'source', 'Id']]


predictions = []

cluster_ids = relevant_data['Cluster ID'].value_counts()
for cluster_id in cluster_ids[cluster_ids>1].index:
    
    fodors_ids = relevant_data[
        (relevant_data['Cluster ID'] == cluster_id) & 
        (relevant_data['source'] == 'fodors')
    ].Id.values
    zagats_ids = relevant_data[
        (relevant_data['Cluster ID'] == cluster_id) & 
        (relevant_data['source'] == 'zagats')
    ].Id.values
    
    match_interim = list(product(fodors_ids, zagats_ids))
    predictions.append(match_interim)
    
m = []
for cluster in predictions: 
    for combo in cluster: 
        m.append([combo[0], combo[1]])
        
predictions = pd.DataFrame(m, columns=['fodors_id', 'zagats_id'])

predictions['f-z'] = predictions.apply(lambda row: f"{row['fodors_id']}-{row['zagats_id']}", axis=1)


predictions.head(10)

In [57]:
results = pd.read_csv('matches_fodors_zagats.csv')
results['f-z'] = results.apply(lambda row: f"{row['fodors_id']}-{row['zagats_id']}", axis=1)


In [58]:
res_set = set(results['f-z'].values.tolist())
pred_set = set(predictions['f-z'].values.tolist())

In [59]:
tp = len(res_set & pred_set)
fn = len(res_set-pred_set)
fp = len(pred_set-res_set)

print(f'tp: {tp} fp: {fp} fn: {fn}')


tp: 111 fp: 4 fn: 1
